In [ ]:
from __future__ import division, print_function
# coding=utf-8
import sys
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras
from skimage import io
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
# Keras
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
# Flask utils
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer
# Define a flask app
app = Flask(__name__)
# Model saved with Keras model.save()
# You can also use pretrained model from Keras
# Check https://keras.io/applications/
model =tf.keras.models.load_model('PlantDNet.h5',compile=False)
print('Model loaded. Check http://127.0.0.1:5000/')
def model_predict(img_path, model):
 img = image.load_img(img_path, grayscale=False, target_size=(64, 64))
 show_img = image.load_img(img_path, grayscale=False, target_size=(64, 64))
 x = image.img_to_array(img)
 x = np.expand_dims(x, axis=0)
 x = np.array(x, 'float32')
 x /= 255
 preds = model.predict(x)
 return preds
@app.route('/', methods=['GET'])
def index():
 # Main page
 return render_template('index.html')
@app.route('/predict', methods=['GET', 'POST'])
def upload():
 if request.method == 'POST':
 # Get the file from post request
 f = request.files['file']
 # Save the file to ./uploads
basepath = os.path.dirname(__file__)
 file_path = os.path.join(
 basepath, 'uploads', secure_filename(f.filename))
 f.save(file_path)
 # Make prediction
 preds = model_predict(file_path, model)
 print(preds[0])
 # x = x.reshape([64, 64]);
 disease_class = ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 
'Potato___Early_blight',
 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 
'Tomato_Early_blight',
 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot',
 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot',
 'Tomato__Tomato_YellowLeaf__Curl_Virus', 
'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']
 a = preds[0]
 ind=np.argmax(a)
 print('Prediction:', disease_class[ind])
 result=disease_class[ind]
 return result
 return None
if __name__ == '__main__':
 # app.run(port=5002, debug=True)
 # Serve the app with gevent
 app.run(port=5000,debug=True)
